In [ ]:
import os
os.chdir('C:/Users/theodore.wong/cif/cities-cif')
import geemap, json
import ee
# ee.Authenticate()
ee.Initialize()
from city_metrix.layers import Layer, EsaWorldCover, EsaWorldCoverClass
from city_metrix.layers.layer import get_image_collection

FILEPATH = 'C:/Users/theodore.wong/danida_africa'

In [ ]:
boundary_filenames = os.listdir('{0}/boundaries'.format(FILEPATH))
boundary_filenames

In [ ]:
class Hand(Layer):
    def __init__(self, spatial_resolution=30, threshold=None, **kwargs):
        super().__init__(**kwargs)
        self.spatial_resolution = spatial_resolution
        self.threshold = threshold

    def get_data(self, bbox):
        dataset = ee.ImageCollection(ee.Image("users/gena/GlobalHAND/30m/hand-1000").focal_mean(0.1))
        if self.threshold is None:
            dataset = ee.ImageCollection(ee.Image("users/gena/GlobalHAND/30m/hand-1000").focal_mean(0.1))
        else:
            dataset = ee.ImageCollection(ee.Image("users/gena/GlobalHAND/30m/hand-1000").focal_mean(0.1).lt(self.threshold))
        hand = ee.ImageCollection(dataset
             .filterBounds(ee.Geometry.BBox(*bbox))
             .select('b1')
             .sum()
             )

        data = get_image_collection(hand, bbox, self.spatial_resolution, "hand")
        return data.b1

In [ ]:
def percent_builtup_in_lowhand(geoms):
    built_up_land = EsaWorldCover(land_cover_class=EsaWorldCoverClass.BUILT_UP)
    lowhand = Hand(threshold=1)
    built_land = built_up_land.groupby(geoms).count()
    built_land_in_lowhand = built_up_land.mask(lowhand).groupby(geoms).count()
    return (built_land_in_lowhand / built_land * 100).round(decimals=1)

In [ ]:
for fname in boundary_filenames:
    if fname.split('-')[3].split('.')[0][-1] != 'n':
        cityname = '{0} {1}'.format(fname.split('-')[1], fname.split('-')[2])
        with open('{0}/boundaries/{1}'.format(FILEPATH, fname), 'r') as ifile:
            json_obj = json.loads(ifile.read())
            feature = geemap.geojson_to_ee(json_obj)
            geoms = geemap.ee_to_gdf(feature)['geometry']
            result = percent_builtup_in_lowhand(geoms)
            out_gdf = geemap.ee_to_gdf(feature)
            out_gdf['percent_builtup_in_lowhand'] = result
            with open('{0}/percent_builtup_in_low_HAND-{1}.geojson'.format(FILEPATH, cityname.replace(' ', '-')), 'w') as ofile:
                ofile.write(out_gdf.to_json())
            print(cityname)